In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

import re
import copy
import nltk
import pickle
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import euclidean, jaccard, hamming

from sklearn.metrics import accuracy_score, mean_squared_error, r2_score, classification_report

In [99]:
# df1 = pd.read_csv("Datasets/instagram_reach.csv")
df1 = pd.read_csv("https://raw.githubusercontent.com/Dhirengit/PPT-Ineuron/main/DataSets/instagram_reach.csv")
df1.sample()

,Unnamed: 0,S.No,USERNAME,Caption,Followers,Hashtags,Time since posted,Likes
22,22,26,onthetopsearch,“No one buys things from Facebook.” “Instagram...,1041,#digitalmarketing #machinelearning#entrepreneu...,2 hours,20


In [100]:
df1.reset_index(drop=True)
df1["Time"] = df1["Time since posted"].apply(lambda x: int(x.split(" ")[0]))
df1.drop(columns=["Unnamed: 0","S.No", "Caption", "USERNAME","Time since posted"], inplace=True)

In [101]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Followers  100 non-null    int64 
 1   Hashtags   100 non-null    object
 2   Likes      100 non-null    int64 
 3   Time       100 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 3.2+ KB


In [102]:
df1.describe()

,Followers,Likes,Time
count,100.00000,100.00000,100.000000
mean,961.96000,46.48000,3.460000
std,1014.62567,55.08698,3.394648
min,11.00000,8.00000,2.000000
25%,252.75000,19.00000,2.000000
50%,612.00000,29.00000,2.000000
75%,1197.00000,46.00000,3.000000
max,4496.00000,349.00000,24.000000


In [103]:
print(df1.loc[:0]["Hashtags"])

0    #MachineLearning #AI #DataAnalytics #DataScien...
Name: Hashtags, dtype: object


In [104]:
def convert_hastag(text):
    ps = PorterStemmer()
    wordnet = WordNetLemmatizer()
    
    tags = nltk.sent_tokenize(text)
    corpus = []
    tag = re.sub("[^a-zA-Z]", " ", text)
    tag = tag.lower()
    tag = tag.split()
#     tag = [wordnet.lemmatize(word) for word in tag if not word in set(stopwords.words("english"))]
    
    return tag

convert_hastag(df1["Hashtags"][0])

['machinelearning', 'ai', 'dataanalytics', 'datascienc', 'datalake']

In [105]:
hastag = pd.DataFrame()
df1["Hashtags"]= df1["Hashtags"].apply(convert_hastag)

In [106]:
print(df1[["Hashtags"]])

                                             Hashtags
0   [machinelearning, ai, dataanalytics, datascien...
1   [deck, mac, macintosh, sayhello, apple, stevej...
2   [whoiswho, aitrading, ai, aitradingteam, insta...
3   [iot, cre, workplace, cdo, bigdata, technology...
4   [instamachinelearning, instabigdata, instamark...
..                                                ...
95  [beverlyhills, realestate, losangelesrealestat...
96  [workspace, work, developer, development, deve...
97  [books, book, motivation, inspiration, life, b...
98  [heavyequipment, underconstruction, dozer, rea...
99  [marketing, programming, development, desarrol...

[100 rows x 1 columns]


In [107]:
unique_hashtag = set(tag for row in df1["Hashtags"] for tag in row)
len(unique_hashtag)

1155

In [108]:
for hashtag in unique_hashtag:
    df1[hashtag] = 0 # initialize 0 value for every hashtag


/tmp/ipykernel_28/1276266570.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1[hashtag] = 0 # initialize 0 value for every hashtag
/tmp/ipykernel_28/1276266570.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1[hashtag] = 0 # initialize 0 value for every hashtag
/tmp/ipykernel_28/1276266570.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a

In [109]:
for index, row in enumerate(df1["Hashtags"]): # enumerate hashtags 
    for tag in row: # Row wise hashtag 
        df1.at[index, tag] = 1 # appply 1 value when index and tag match 


In [110]:
df1.drop(columns=["Hashtags"], inplace=True)

In [111]:
x1 = df1.drop(columns=["Likes", "Time"])

In [112]:
y1 = df1[["Likes", "Time"]]

In [113]:
x1_train, x1_test, y1_train, y1_test = train_test_split(x1, y1, test_size=0.1)

In [114]:
print(f"X_train shape = {x1_train.shape}")
print(f"X_test shape = {x1_test.shape}")
print(f"y_train shape = {y1_train.shape}")
print(f"y_test shape = {y1_test.shape}")

X_train shape = (90, 1156)
X_test shape = (10, 1156)
y_train shape = (90, 2)
y_test shape = (10, 2)


In [115]:
y1_test

,Likes,Time
97,10,3
72,20,2
70,21,2
34,273,2
46,24,2
47,51,3
86,43,4
79,29,2
40,24,2
0,139,11


In [116]:
rf = RandomForestRegressor()

In [117]:
rf.fit(x1_train, y1_train)

RandomForestRegressor()

In [118]:
y1_pred = rf.predict(x1_test)
y1_pred

array([[ 16.17      ,   2.18      ],
       [ 31.36      ,   2.43      ],
       [ 26.17      ,   2.39      ],
       [ 16.13      ,   2.13      ],
       [ 44.82      ,   3.9       ],
       [109.01      ,   4.14      ],
       [ 35.65      ,   2.67666667],
       [ 33.78      ,   2.69666667],
       [ 19.14      ,   2.26      ],
       [ 63.11      ,   6.09      ]])

In [119]:
mse1 = mean_squared_error(y1_test, y1_pred)
mse1

3793.340377777778

In [120]:
rmse1 = np.sqrt(mse1)
rmse1

61.590099673387265

In [121]:
r2score1 = r2_score(y1_test, y1_pred)
r2score1

0.14806848117175264

In [122]:
# df2 = pd.read_csv("DataSets/ObesityDataSet_raw_and_data_sinthetic.csv")
df2 = pd.read_csv("https://raw.githubusercontent.com/Dhirengit/PPT-Ineuron/main/DataSets/ObesityDataSet_raw_and_data_sinthetic.csv")

In [123]:
df2.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


In [124]:
df2.isnull().sum()

Gender                            0
Age                               0
Height                            0
Weight                            0
family_history_with_overweight    0
FAVC                              0
FCVC                              0
NCP                               0
CAEC                              0
SMOKE                             0
CH2O                              0
SCC                               0
FAF                               0
TUE                               0
CALC                              0
MTRANS                            0
NObeyesdad                        0
dtype: int64

In [125]:
df2.describe()

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
count,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000
mean,24.312600,1.701677,86.586058,2.419043,2.685628,2.008011,1.010298,0.657866
std,6.345968,0.093305,26.191172,0.533927,0.778039,0.612953,0.850592,0.608927
min,14.000000,1.450000,39.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,19.947192,1.630000,65.473343,2.000000,2.658738,1.584812,0.124505,0.000000
50%,22.777890,1.700499,83.000000,2.385502,3.000000,2.000000,1.000000,0.625350
75%,26.000000,1.768464,107.430682,3.000000,3.000000,2.477420,1.666678,1.000000
max,61.000000,1.980000,173.000000,3.000000,4.000000,3.000000,3.000000,2.000000


In [126]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          2111 non-null   object 
 1   Age                             2111 non-null   float64
 2   Height                          2111 non-null   float64
 3   Weight                          2111 non-null   float64
 4   family_history_with_overweight  2111 non-null   object 
 5   FAVC                            2111 non-null   object 
 6   FCVC                            2111 non-null   float64
 7   NCP                             2111 non-null   float64
 8   CAEC                            2111 non-null   object 
 9   SMOKE                           2111 non-null   object 
 10  CH2O                            2111 non-null   float64
 11  SCC                             2111 non-null   object 
 12  FAF                             21

In [127]:
df2.sample(5)

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
339,Female,19.000000,1.530000,42.000000,no,no,2.000000,3.000000,Sometimes,no,1.000000,yes,2.000000,0.000000,Frequently,Public_Transportation,Insufficient_Weight
230,Male,20.000000,1.900000,91.000000,yes,yes,3.000000,4.000000,Sometimes,no,3.000000,no,2.000000,0.000000,Sometimes,Walking,Overweight_Level_I
205,Female,23.000000,1.600000,78.000000,yes,yes,2.000000,1.000000,Sometimes,yes,2.000000,no,1.000000,0.000000,Frequently,Public_Transportation,Obesity_Type_I
1128,Male,30.022598,1.747739,83.314157,yes,yes,2.011656,3.165837,Sometimes,no,1.844645,no,0.889963,0.395979,no,Automobile,Overweight_Level_II
1217,Female,22.654316,1.621233,82.000000,yes,yes,1.063449,1.000000,Sometimes,no,2.000000,no,0.000000,1.482016,Sometimes,Public_Transportation,Obesity_Type_I


In [128]:
df2["SCC"].unique()

array(['no', 'yes'], dtype=object)

In [129]:
df2[["Age", "Weight"]] = df2[["Age", "Weight"]].astype(int)

In [130]:
df2["Gender"] = df2["Gender"].apply(lambda x : 1 if x == "Male" else 0)

In [131]:
df2["SMOKE"] = df2["SMOKE"].apply(lambda x : 1 if x == "yes" else 0)

In [132]:
df2["family_history_with_overweight"] = df2["family_history_with_overweight"].apply(lambda x : 1 if x == "yes" else 0)

In [133]:
df2["FAVC"] = df2["FAVC"].apply(lambda x : 1 if x == "yes" else 0)

In [134]:
df2["SCC"] = df2["SCC"].apply(lambda x : 1 if x == "yes" else 0)

In [135]:
df2

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,21,1.620000,64,1,0,2.0,3.0,Sometimes,0,2.000000,0,0.000000,1.000000,no,Public_Transportation,Normal_Weight
1,0,21,1.520000,56,1,0,3.0,3.0,Sometimes,1,3.000000,1,3.000000,0.000000,Sometimes,Public_Transportation,Normal_Weight
2,1,23,1.800000,77,1,0,2.0,3.0,Sometimes,0,2.000000,0,2.000000,1.000000,Frequently,Public_Transportation,Normal_Weight
3,1,27,1.800000,87,0,0,3.0,3.0,Sometimes,0,2.000000,0,2.000000,0.000000,Frequently,Walking,Overweight_Level_I
4,1,22,1.780000,89,0,0,2.0,1.0,Sometimes,0,2.000000,0,0.000000,0.000000,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,0,20,1.710730,131,1,1,3.0,3.0,Sometimes,0,1.728139,0,1.676269,0.906247,Sometimes,Public_Transportation,Obesity_Type_III
2107,0,21,1.748584,133,1,1,3.0,3.0,Sometimes,0,2.005130,0,1.341390,0.599270,Sometimes,Public_Transportation,Obesity_Type_III
2108,0,22,1.752206,133,1,1,3.0,3.0,Sometimes,0,2.054193,0,1.414209,0.646288,Sometimes,Public_Transportation,Obesity_Type_III
2109,0,24,1.739450,133,1,1,3.0,3.0,Sometimes,0,2.852339,0,1.139107,0.586035,Sometimes,Public_Transportation,Obesity_Type_III


In [136]:
df2["MTRANS"].unique()

array(['Public_Transportation', 'Walking', 'Automobile', 'Motorbike',
       'Bike'], dtype=object)

In [137]:
label_encoder = LabelEncoder()

In [138]:
df2["CAEC"] =label_encoder.fit_transform(df2["CAEC"])

In [139]:
df2["CALC"] = label_encoder.fit_transform(df2["CALC"])

In [140]:
df2["MTRANS"] = label_encoder.fit_transform(df2["MTRANS"])

In [141]:
df2["NObeyesdad"] = label_encoder.fit_transform(df2["NObeyesdad"])

In [142]:
df2.sample(5)

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
607,1,18,1.753389,54,1,1,2.000000,3.166450,2,0,2.278578,0,1.838881,2.000000,2,3,0
629,0,22,1.751691,55,1,1,2.478891,3.371832,1,0,2.004126,0,2.000000,0.327376,3,3,0
1312,1,31,1.676595,89,1,1,2.934671,2.119682,2,0,2.041462,0,0.578074,0.000000,2,0,2
1775,1,37,1.762921,118,1,1,2.136830,2.993084,2,0,1.885926,0,0.615298,0.000000,2,0,3
69,1,23,1.800000,60,1,0,2.000000,3.000000,3,0,3.000000,0,0.000000,1.000000,2,3,1


In [143]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          2111 non-null   int64  
 1   Age                             2111 non-null   int64  
 2   Height                          2111 non-null   float64
 3   Weight                          2111 non-null   int64  
 4   family_history_with_overweight  2111 non-null   int64  
 5   FAVC                            2111 non-null   int64  
 6   FCVC                            2111 non-null   float64
 7   NCP                             2111 non-null   float64
 8   CAEC                            2111 non-null   int64  
 9   SMOKE                           2111 non-null   int64  
 10  CH2O                            2111 non-null   float64
 11  SCC                             2111 non-null   int64  
 12  FAF                             21

# 4 Different Classification algorithms

In [144]:
x2 = df2.drop(columns=["NObeyesdad"])
y2 = df2["NObeyesdad"]

In [145]:
y2

0       1
1       1
2       1
3       5
4       6
       ..
2106    4
2107    4
2108    4
2109    4
2110    4
Name: NObeyesdad, Length: 2111, dtype: int64

In [146]:
x2_train, x2_test, y2_train, y2_test = train_test_split(x2,y2, test_size=0.2)

In [147]:
log_r2 = LogisticRegression(solver="liblinear")

In [148]:
log_r2.fit(x2_train, y2_train)

LogisticRegression(solver='liblinear')

In [149]:
y2_pred_log = log_r2.predict(x2_test)

In [150]:
accuracy_score(y2_test, y2_pred_log)

0.7210401891252955

In [151]:
knn2 = KNeighborsClassifier()

In [152]:
knn2.fit(x2_train, y2_train)

KNeighborsClassifier()

In [153]:
y2_pred_knn = knn2.predict(x2_test)

In [154]:
accuracy_score(y2_test, y2_pred_knn)

0.8628841607565012

In [155]:
svm2 = svm.SVC(kernel="linear")

In [156]:
svm2.fit(x2_train, y2_train)

SVC(kernel='linear')

In [157]:
y2_pred_svm = svm2.predict(x2_test)

In [158]:
accuracy_score(y2_test, y2_pred_svm)

0.8581560283687943

In [159]:
naive2 = MultinomialNB()

In [160]:
naive2.fit(x2_train, y2_train)

MultinomialNB()

In [161]:
y2_pred_naive = naive2.predict(x2_test)

In [162]:
accuracy_score(y2_test, y2_pred_naive)

0.541371158392435

In [163]:
# df_3 =  pd.read_json("DataSets/News_Category_Dataset_v3.json", lines=True)

df_3 =  pd.read_json("https://raw.githubusercontent.com/Dhirengit/PPT-Ineuron/main/DataSets/News_Category_Dataset_v3.json", lines=True)

In [164]:
df_3.shape

(209527, 6)

In [165]:
df_3_new = copy.deepcopy(df_3)

In [166]:
df_3_new = df_3.sample(1100)
df_3_new.shape

(1100, 6)

In [167]:
def link_filter(text):
    replace_text = text.replace("https://www.huffpost.com/entry/", "")
    replace_text = replace_text.replace("https://www.huffingtonpost.com/entry/", "")
    replace_text = replace_text.split("_")[0]
    replace_text = re.sub("[^a-zA-Z]", " ",replace_text )
    return replace_text

In [168]:
df_3_new["link"]= df_3["link"].apply(link_filter)

In [169]:
df_3_new.shape

(1100, 6)

In [170]:
def text_preprecess(text, use_stop_word=True):
    ps = PorterStemmer()
#     wordnet = WordNetLemmatizer()
    
    tags = nltk.sent_tokenize(text)
    tag = re.sub("[^a-zA-Z]", " ", text)
    tag = tag.lower()
    tag = tag.split()
    tag = [ps.stem(word) for word in tag if not word in set(stopwords.words("english"))] if use_stop_word else tag
    
    return tag

In [171]:
df_3_new["link"] = df_3_new["link"].apply(text_preprecess)

In [172]:
df_3_new["headline"] = df_3_new["headline"].apply(text_preprecess)

In [173]:
df_3_new["short_description"] = df_3_new["short_description"].apply(text_preprecess)

In [174]:
df_3_new["authors"] = df_3_new["authors"].apply(text_preprecess, use_stop_word=False)

In [175]:
df_3_new["category"] = df_3_new["category"].apply(text_preprecess, use_stop_word=False)

In [176]:
df_3_new["tags"] = df_3_new["link"] + df_3_new["headline"] + df_3_new["short_description"] + df_3_new["category"] + df_3_new["authors"]

In [177]:
df_3_new.drop(columns=["link", "headline", "short_description", "category", "authors","date"], inplace=True)

In [178]:
df_3_new["tags"] = df_3_new["tags"].apply(lambda text : " ".join(text))

In [179]:
df_3_new.shape

(1100, 1)

In [180]:
df_3_new

,tags
118047,american educ test matter test score stop obse...
171750,girl stem mentorship girl stem call mentor tog...
68483,radiohead drop new singl burn witch onlin radi...
180974,uga footbal script murray determin championshi...
122097,kendal jenner text vma kendal jenner address v...
...,...
68346,chloe grace moretz throw around lot buzzword a...
79723,peni pump exchang pistol man pull gun clerk fa...
147965,mental health new psychiatri forget everyth th...
65560,daili medit spiritu susten daili medit spiritu...


In [181]:
cv3 = CountVectorizer(max_features=1500)

In [182]:
vector = cv3.fit_transform(df_3_new["tags"]).toarray()
vector.shape

(1100, 1500)

In [183]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)

In [184]:
similarity

array([[1.        , 0.07537784, 0.07756315, ..., 0.09656091, 0.        ,
        0.05330018],
       [0.07537784, 1.        , 0.08574929, ..., 0.02668803, 0.        ,
        0.        ],
       [0.07756315, 0.08574929, 1.        , ..., 0.10984701, 0.        ,
        0.        ],
       ...,
       [0.09656091, 0.02668803, 0.10984701, ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.05330018, 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [185]:
# def euclidean_distance(point1, point2):
#     return np.linalg.norm(point1 - point2)

# # Function to find the most similar data point using Euclidean distance
# def find_most_similar_data(dataset, given_data):
#     distances = [euclidean_distance(given_data, data) for data in dataset]
#     most_similar_index = np.argmin(distances)
#     return dataset[most_similar_index]


# # Example usage:
# given_data = vector[0]  # The point for which we want to find the most similar point
# most_similar_data = find_most_similar_data(vector, given_data)

# print("Given data point:", given_data, vector.shape)
# print("Most similar data point:", most_similar_data, given_data.shape)

In [186]:
data_array = vector

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

max_similarity = -1
most_similar_pair = (0, 0)

for i in range(len(data_array)):
    for j in range(i + 1, len(data_array)):
        similarity = cosine_similarity(data_array[i], data_array[j])
        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_pair = (i, j)
            print(i,j,"max_similarity", max_similarity)


print("Cosine Most similar pair:", most_similar_pair)
print("Cosine Similarity score:", max_similarity)

0 1 max_similarity 0.07537783614444091
0 2 max_similarity 0.07756315349925287
0 64 max_similarity 0.21661214442955293
0 808 max_similarity 0.43684887642861214
9 35 max_similarity 0.5863527298559492
37 241 max_similarity 0.5936657514041415
41 183 max_similarity 0.653358607776565
41 216 max_similarity 0.6634888026970371
62 117 max_similarity 0.6842105263157894
62 848 max_similarity 0.7029594915666377
68 782 max_similarity 0.7249999999999999
117 848 max_similarity 0.7570332986102252
Cosine Most similar pair: (117, 848)
Cosine Similarity score: 0.7570332986102252


In [187]:
df_3_new.iloc[most_similar_pair[0]], df_3_new.iloc[most_similar_pair[1]]

(tags    maggi hutchason gp guid maggi hutchason gp gui...
 Name: 74017, dtype: object,
 tags    ronda lacroix gp guid ronda lacroix gp guid le...
 Name: 72807, dtype: object)

In [188]:
max_similarity= -1
most_similar_pair = (0, 0)

for i in range(len(data_array)):
    for j in range(i + 1, len(data_array)):
        distance = euclidean(data_array[i], data_array[j])
        similarity = 1 / (1 + distance)
        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_pair = (i, j)
            print(i,j,"max_similarity", max_similarity)
     
print("Euclidean Most Similar pair:", most_similar_pair)
print("Euclidean Similarity score:", max_similarity)

0 1 max_similarity 0.10414144201428256
0 2 max_similarity 0.10542649822871226
0 3 max_similarity 0.11268865367232477
0 4 max_similarity 0.12178632452799958
0 16 max_similarity 0.12389934309929541
0 139 max_similarity 0.125
0 208 max_similarity 0.1261319836228832
0 878 max_similarity 0.12729683913915313
1 4 max_similarity 0.13100580420257674
1 16 max_similarity 0.13367660240019172
1 208 max_similarity 0.1365270594958143
1 878 max_similarity 0.1380262631157473
2 208 max_similarity 0.13957875683699938
2 404 max_similarity 0.14285714285714285
3 4 max_similarity 0.1639607805437114
3 16 max_similarity 0.1757340838011157
3 591 max_similarity 0.1827439976315568
3 878 max_similarity 0.1951941016011038
4 16 max_similarity 0.21089672205953397
4 208 max_similarity 0.2240092377397959
4 878 max_similarity 0.23166247903554
16 208 max_similarity 0.2402530733520421
16 878 max_similarity 0.25
20 25 max_similarity 0.27429188517743175
1036 1044 max_similarity 0.28989794855663564
Euclidean Most Similar pai

In [189]:
df_3_new.iloc[most_similar_pair[0]], df_3_new.iloc[most_similar_pair[1]]

(tags    bailey dessert recip bailey dessert recip phot...
 Name: 171371, dtype: object,
 tags    bruschetta recip bruschetta crostini recip way...
 Name: 154750, dtype: object)

In [190]:
#Jaccard Similarity
def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    similarity = intersection / union
    return similarity

most_similar_row = None
max_similarity = -1

for i in range(data_array.shape[0]):
    for j in range(i+1 , data_array.shape[0]):
        jaccard_similarity = 1 - jaccard(data_array[i], data_array[j])
        if jaccard_similarity > max_similarity:
            max_similarity = jaccard_similarity
            most_similar_row = i, j
            print(i,j, max_similarity)
            

# Print the indices of the most similar rows and their Jaccard similarity score
print(f"Jaccard Most similar rows: {most_similar_row[0]} and {most_similar_row[1]}")
print(f"Jaccard similarity score: {max_similarity}")

0 1 0.09090909090909094
0 249 0.09302325581395354
0 763 0.09999999999999998
0 819 0.10344827586206895
1 559 0.11111111111111116
2 329 0.125
4 149 0.16666666666666663
4 496 0.2222222222222222
4 646 0.25
18 90 0.2692307692307693
36 461 0.2727272727272727
37 241 0.2962962962962963
45 289 0.3076923076923077
45 1044 0.4
62 117 0.4375
62 848 0.4666666666666667
117 848 0.47058823529411764
1036 1044 0.625
Jaccard Most similar rows: 1036 and 1044
Jaccard similarity score: 0.625


In [191]:
df_3_new.iloc[most_similar_row[0]], df_3_new.iloc[most_similar_row[1]]

(tags    bailey dessert recip bailey dessert recip phot...
 Name: 171371, dtype: object,
 tags    bruschetta recip bruschetta crostini recip way...
 Name: 154750, dtype: object)

**correlation between Rows**

In [193]:
from scipy.spatial.distance import euclidean, jaccard, hamming, correlation

In [203]:
correlation(data_array[1], data_array[1])

0.0

In [207]:
most_similar_row = None
max_correlation = -1

for i in range(data_array.shape[0]):
    for j in range(i+1 , data_array.shape[0]):
        corr = 1 - correlation(data_array[i], data_array[j])
        if corr > max_correlation:
            max_correlation = corr
            most_similar_row = i, j
            print(i,j, max_correlation)
            

# Print the indices of the most similar rows and their Jaccard similarity score
print(f"Most correlation similar rows: {most_similar_row[0]} and {most_similar_row[1]}")
print(f"correlation similarity score: {max_correlation}")

df_3_new.iloc[most_similar_row[0]], df_3_new.iloc[most_similar_row[1]]

0 1 0.0675395757752667
0 2 0.0704368634312007
0 64 0.20906268151013174
0 808 0.4306051840817928
9 35 0.5840300542665204
37 241 0.5901806862965344
41 183 0.6509584812121509
41 216 0.661026778441497
62 117 0.6820835051740806
62 848 0.7010216085909317
68 782 0.7236056945949022
117 848 0.7551850396091573
Most correlation similar rows: 117 and 848
correlation similarity score: 0.7551850396091573


(tags    maggi hutchason gp guid maggi hutchason gp gui...
 Name: 74017, dtype: object,
 tags    ronda lacroix gp guid ronda lacroix gp guid le...
 Name: 72807, dtype: object)

In [ ]:
# import Levenshtein
# def levenshtein_similarity(row1, row2):
#     return 1 - (Levenshtein.distance(row1, row2) / max(len(row1), len(row2)))

# most_similar_row = None
# max_similarity = -1

# for i in range(data_array.shape[0]):
#     for j in range(i + 1, data_array.shape[0]):
#         similarity = levenshtein_similarity(data_array[i], data_array[j])
#         if similarity > max_similarity:
#             max_similarity = similarity
#             most_similar_row = (i, j)
#             print(i,j, max_similarity)

# print(f"Most similar rows: {most_similar_row}")
# print(f"Similarity score: {max_similarity}")

In [2]:
df_4 = pd.read_csv("DataSets/online_shoppers_intention.csv")

In [4]:
df_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Administrative           12330 non-null  int64  
 1   Administrative_Duration  12330 non-null  float64
 2   Informational            12330 non-null  int64  
 3   Informational_Duration   12330 non-null  float64
 4   ProductRelated           12330 non-null  int64  
 5   ProductRelated_Duration  12330 non-null  float64
 6   BounceRates              12330 non-null  float64
 7   ExitRates                12330 non-null  float64
 8   PageValues               12330 non-null  float64
 9   SpecialDay               12330 non-null  float64
 10  Month                    12330 non-null  object 
 11  OperatingSystems         12330 non-null  int64  
 12  Browser                  12330 non-null  int64  
 13  Region                   12330 non-null  int64  
 14  TrafficType           